# NEPOOL IDR Drop

Implements class to batch tasks used for IDR drops.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
import json
import os

## Open Raw .json

open a local .json file of email body info.

In [ ]:
with open('bodies.json', 'r') as f:
    bodies = json.load(f)

bodies = json.loads(bodies)

#print(bodies.keys())
for item in bodies.values():
    print(item['accts'])

## Download files from EPO portal

In [3]:
browser, url = EPOwebscrape.logon('stopshop-engie', '3822b', False)

user:  stopshop-engie
password:  3822b
logging on...
set to last  400  days.


In [4]:
print('(browser, url) outputs type: {}.'.format((type(browser), type(url))))

(browser, url) outputs type: (<class 'selenium.webdriver.chrome.webdriver.WebDriver'>, <class 'str'>).


In [6]:
url

'https://eversource.epo.schneider-electric.com/eversource/cgi/eponline.exe'

In [5]:
#browser, url = logon('somer-nengie', 'rb4171', False)

browser, url = EPOwebscrape.logon('stopshop-engie', '3822b', False)

accts_to_find = ['0040677012', '0021483009']
#accts_to_find = ['0040677012', '0021483009', '0067880031', '0074816023'] #enter accts as list
#accts_to_find = ['28905980018', '26121881010']
AIDs = []

soup = BeautifulSoup(browser.page_source)
table = soup.find('tbody', {'role' : 'rowgroup'})

#get EPO AID value for every account
for accts in accts_to_find:
    results = EPOwebscrape.big_match(accts, table)
    AIDs.append(results)

#make list of AIDs, split into list of lists of 5
final = []
AIDs

for aid_list in AIDs:
    for aid in aid_list:
        final.append(aid)

n = 5
final2 = [final[i * n:(i + 1) * n] for i in range((len(final) + n - 1) // n )]  
final2

for elem in final2:
    EPOwebscrape.export_data(elem, browser)


user:  stopshop-engie
password:  3822b
logging on...
set to last  400  days.


# Show downloaded files from EPO portal

Here *filepath* is a directory containing downloaded EPO files. Code will print 20 most recent files.

In [8]:
#filepath = os.getcwd()
readpath = 'C:\\Users\\wb5888\\Downloads'

myfiles = IDRdrop.show_dir(readpath, 30)
print(myfiles)

files found in dir:  C:\Users\wb5888\Downloads
                                                files                time
0   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:14:03
1   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:13:16
2   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:12:45
3   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:12:18
4   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:11:37
5   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:11:08
6   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:10:33
7   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:09:54
8   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:09:19
9   eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:08:40
10  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 14:08:02
11  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e5... 2019-07-22 

Choose files to split into Raw IDR files.

In [9]:
index = [0]

splitfiles = list(myfiles.files[:20])
print('files to split: ')
print(splitfiles)

files to split: 
['eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_c8c6e9cc_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_7ed9d2d0_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_b84e022a_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_f2de76df_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_5a459304_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_ffab1d9c_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_634bccc1_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_3680b926_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_bf8ac328_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_826b46a4_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_75babe02_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_db98444a_hourlycsv.csv', 'eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_c5904ef6_hourlycsv.csv', 'eversource_9e1c18d8

## Batch process downloaded EPO files into Raw IDRs

- track overall progress

In [10]:
readpath = 'C:\\Users\\wb5888\\Downloads'
writepath = 'C:\\Users\\wb5888\\Documents\\Raw IDR Data'
error_log = []

for file in splitfiles:
    try:
        os.chdir(readpath)
        filedf = pd.read_csv(file, sep = ",", header = 0)
    
        IDRdrop.raw_split(filedf, readpath, writepath)
        print('success, file: ', file)
        
    except:
        error_log = error_log.append(file)
        print('error, file: ', file)

found 2 accounts.
51828534040_191982001_CLP_IDR_RAW.csv  written.
success, file:  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_c8c6e9cc_hourlycsv.csv
found 4 accounts.
51174223057_961712008_CLP_IDR_RAW.csv  written.
51410742019_399071000_CLP_IDR_RAW.csv  written.
success, file:  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_7ed9d2d0_hourlycsv.csv
found 4 accounts.
51164463044_165442005_CLP_IDR_RAW.csv  written.
51179642020_370171001_CLP_IDR_RAW.csv  written.
51287942056_514271007_CLP_IDR_RAW.csv  written.
51749603031_564702002_CLP_IDR_RAW.csv  written.
success, file:  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_b84e022a_hourlycsv.csv
found 4 accounts.
51251134078_850682009_CLP_IDR_RAW.csv  written.
51325004083_233662004_CLP_IDR_RAW.csv  written.
51455013003_558902004_CLP_IDR_RAW.csv  written.
51483603056_147702007_CLP_IDR_RAW.csv  written.
success, file:  eversource_9e1c18d8-6a01-435e-b946-e49ce8158e56_f2de76df_hourlycsv.csv
found 4 accounts.
51143743037_393332008_CLP_IDR_RAW.

## Show Raw IDR files based on utility

In [11]:
utility = "CLP"

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [12]:
rawpath = 'C:\\Users\\wb5888\\Documents\\Raw IDR Data'

rawfiles = IDRdrop.show_dir(rawpath, 25)
print(rawfiles)

files found in dir:  C:\Users\wb5888\Documents\Raw IDR Data
                                    files                time
0   51475742045_628171006_CLP_IDR_RAW.csv 2019-07-22 14:15:33
1   51478542046_182071007_CLP_IDR_RAW.csv 2019-07-22 14:15:33
2   51276642014_290171005_CLP_IDR_RAW.csv 2019-07-22 14:15:32
3   51017593005_637362006_CLP_IDR_RAW.csv 2019-07-22 14:15:31
4   51746053099_576632000_CLP_IDR_RAW.csv 2019-07-22 14:15:30
5   51432034007_749482008_CLP_IDR_RAW.csv 2019-07-22 14:15:29
6   51749524039_175282000_CLP_IDR_RAW.csv 2019-07-22 14:15:29
7   51694943077_678432008_CLP_IDR_RAW.csv 2019-07-22 14:15:28
8   51817652050_047671001_CLP_IDR_RAW.csv 2019-07-22 14:15:27
9   51271824005_392482006_CLP_IDR_RAW.csv 2019-07-22 14:15:26
10  51821104080_034762001_CLP_IDR_RAW.csv 2019-07-22 14:15:25
11  51473993061_726562002_CLP_IDR_RAW.csv 2019-07-22 14:15:24
12  51793024050_188972001_CLP_IDR_RAW.csv 2019-07-22 14:15:23
13  51429803075_715012006_CLP_IDR_RAW.csv 2019-07-22 14:15:23
14  511777

Choose Raw IDRs to filter into IDR files.

In [16]:
index = [0]

processfiles = rawfiles.files[:22]
files = list(processfiles)
print('files to processed: ')
print(processfiles)

files to processed: 
0     51475742045_628171006_CLP_IDR_RAW.csv
1     51478542046_182071007_CLP_IDR_RAW.csv
2     51276642014_290171005_CLP_IDR_RAW.csv
3     51017593005_637362006_CLP_IDR_RAW.csv
4     51746053099_576632000_CLP_IDR_RAW.csv
5     51432034007_749482008_CLP_IDR_RAW.csv
6     51749524039_175282000_CLP_IDR_RAW.csv
7     51694943077_678432008_CLP_IDR_RAW.csv
8     51817652050_047671001_CLP_IDR_RAW.csv
9     51271824005_392482006_CLP_IDR_RAW.csv
10    51821104080_034762001_CLP_IDR_RAW.csv
11    51473993061_726562002_CLP_IDR_RAW.csv
12    51793024050_188972001_CLP_IDR_RAW.csv
13    51429803075_715012006_CLP_IDR_RAW.csv
14    51177793080_630562001_CLP_IDR_RAW.csv
15    51258474089_608592006_CLP_IDR_RAW.csv
16    51252493093_757262004_CLP_IDR_RAW.csv
17    51672453040_066832001_CLP_IDR_RAW.csv
18    51647153097_478632008_CLP_IDR_RAW.csv
19    51481942043_595171004_CLP_IDR_RAW.csv
20    51132503046_871702007_CLP_IDR_RAW.csv
21    51566893020_713662001_CLP_IDR_RAW.csv
Name: files

## Batch filter Raw IDR into IDR files to be dropped

In [21]:
readpath = rawpath
writepath = 'C:\\Users\\wb5888\\Documents\\IDR Data\\NEPOOL\\'
error_log = []

for dropfile in files:
    try:
        IDRdrop.data_drop(dropfile, rawpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        try:
            error_log = error_log.append(dropfile)
            print("error, file: ", dropfile)
        except:
            pass

success, file:  51475742045_628171006_CLP_IDR_RAW.csv
success, file:  51478542046_182071007_CLP_IDR_RAW.csv
success, file:  51276642014_290171005_CLP_IDR_RAW.csv
success, file:  51017593005_637362006_CLP_IDR_RAW.csv
spot check output file at date  4/10/2019
success, file:  51746053099_576632000_CLP_IDR_RAW.csv
error, file:  51432034007_749482008_CLP_IDR_RAW.csv
spot check output file at date  4/23/2019
success, file:  51749524039_175282000_CLP_IDR_RAW.csv
spot check output file at date  1/1/2019
success, file:  51694943077_678432008_CLP_IDR_RAW.csv
spot check output file at date  1/1/2019
success, file:  51817652050_047671001_CLP_IDR_RAW.csv
spot check output file at date  5/22/2019
success, file:  51271824005_392482006_CLP_IDR_RAW.csv
success, file:  51821104080_034762001_CLP_IDR_RAW.csv
spot check output file at date  5/21/2019
success, file:  51473993061_726562002_CLP_IDR_RAW.csv
spot check output file at date  3/20/2019
success, file:  51177793080_630562001_CLP_IDR_RAW.csv
spot che

In [24]:
type(error_log)
error_log

NoneType